<a href="https://colab.research.google.com/github/lev4ek0/texts/blob/main/Task_2_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Информационный поиск

Скачиваем классический набор данных -- набор текстов об аэронавтике CRANFIELD

In [1]:
! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
! tar -xvf cran.tar.gz
! rm cran.tar.gz*

cran.all.1400
cran.qry
cranqrel
cranqrel.readme


Берём только сами запросы (это будут наши документы)

In [2]:
! grep -v "^\." cran.qry > just.qry
! head -3 just.qry

what similarity laws must be obeyed when constructing aeroelastic models
of heated high speed aircraft .
what are the structural and aeroelastic problems associated with flight


Объединяем многострочные в один

In [3]:
raw_query_data = [line.strip() for line in open("just.qry", "r").readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

query_data[:2] #Выведем пару документов для примера

['what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft . ',
 'what are the structural and aeroelastic problems associated with flight of high speed aircraft . ']

### Составим запросы к нашим документам

In [14]:
QUERIES = ['pressure distributions', 'non-circular cylinders']


## Boolean retrieval
Представим каждый документ как "битовую маску": вектор размером со словарь, в котором на каждой позиции единица, если в документе есть соответсвующий терм, и ноль, если терма нет

In [5]:
# в разных версиях ответы могут отличаться, поэтому важно иметь одну и ту же
! pip install -q scikit-learn==0.22.2.post1

In [15]:
from  sklearn.feature_extraction.text import CountVectorizer

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)
list(encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

Посмотрим на представление первого предложения

In [16]:
id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]
terms

['what',
 'similarity',
 'laws',
 'must',
 'be',
 'obeyed',
 'when',
 'constructing',
 'aeroelastic',
 'models',
 'of',
 'heated',
 'high',
 'speed',
 'aircraft']

Всё так.

## Задание 0

Теперь для каждого из данных запросов `QUERIES` найдём ближайший для него документ из `query_data` по сходству Жаккара. Есть более эффективные способы это сделать, но вам требуется реализовать расстояние Жаккара и далее применить его к нашим данным.

In [8]:
import numpy as np 

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  """
    Сходство или коэффициент Жаккара: отношение мощности пересечения
    к мощности объединения
  """
  count1 = 0
  count2 = 0
  for i, j in zip(vector_a, vector_b):
    if i + j == 2:
      count1 += 1
    if i + j != 0:
      count2 += 1
  return count1/count2

#Проверка, что функция работает правильно
assert jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1])) == 0.4

Здесь документы представлены так же, как строки в матрице термов-документов. Каждая ячейка вектора отвечает за наличие/отсутствие конкретного элемента (например, слова-терма, когда у нас в словаре всего 5 слов). В первом случае их три, во втором — четыре. Объединение — все пять возможных элементов. Пересечение — два. Отсюда и 0.4.

## Задание 1
Теперь с помощью кода ниже вычислите для каждого запроса самые близкие документы.

In [17]:
for q_id, query in enumerate(encoded_queries):
  # приводим к нужному типу
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  # вычисляем коэфф. Жаккара
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  # сортируем по нему
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  # выводим по 3 наиболее близких документа для каждого запроса
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))



Q: pressure distributions
FOUND:
    204	0.29	do viscous effects seriously modify pressure distributions . 
    17	0.17	are experimental pressure distributions on bodies of revolution at angle of attack available . 
    202	0.12	how accurate are existing analytical theories in estimating pressure distributions on cones at incidence,  at hypersonic speeds . 
Q: non-circular cylinders
FOUND:
    58	0.25	how much is known about boundary layer flows along non-circular cylinders . 
    195	0.15	in the problem of the buckling strength of uniform circular cylinders loaded in axial compression,  does the linear solution help with improving the non-linear one . 
    57	0.12	is it possible to determine rates of forced convective heat transfer from heated cylinders of non-circular cross-section,  (the fluid flow being along the generators) . 


Видим, что кое-где просачиваются  тексты, которых с запросами объединяют малозначительные термы, но при этом коэффициент Жаккара -- наша функция ранжирования! -- высок.

# VSM

Попробуем теперь сделать то же, но с tf-idf и косинусным расстоянием. Мы сделаем всё опять "руками", но "в реальной жизни" лучше использоватьесть эффективные реализации cosine distance, например, из библиотеки scipy.

In [18]:
from  sklearn.feature_extraction.text import TfidfVectorizer

# Совет: обязательно разберитесь с тем, какие возможности
# предоставляет tf-idf vectorizer, какие параметры за что отвечают

tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

list(tfidf_encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

## Задание 2
Реализовать косинусное расстояние

In [19]:
import numpy as np 
from scipy.spatial.distance import cosine

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  """
    Косинусное расстояние: единица минус отношение скалярного произведения
    на произведение L2-норм (подсказка: в numpy такие нормы есть)
  """
  return cosine(vector_a, vector_b)
#Проверка, что функция работает правильно
assert cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])) == 0.5


Теперь вычислим ближайшие по косинусному расстоянию между векторными представлениями документов и запросов

In [20]:
for q_id, query in enumerate(tfidf_encoded_queries):
  
  # приводим к нужному типу
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  # Косинусное расстояние
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) \
                       for doc_id, doc in enumerate(docs)]
  # сортируем по нему
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: pressure distributions
FOUND:
    204	0.52	do viscous effects seriously modify pressure distributions . 
    17	0.58	are experimental pressure distributions on bodies of revolution at angle of attack available . 
    202	0.69	how accurate are existing analytical theories in estimating pressure distributions on cones at incidence,  at hypersonic speeds . 
Q: non-circular cylinders
FOUND:
    58	0.51	how much is known about boundary layer flows along non-circular cylinders . 
    195	0.67	in the problem of the buckling strength of uniform circular cylinders loaded in axial compression,  does the linear solution help with improving the non-linear one . 
    57	0.69	is it possible to determine rates of forced convective heat transfer from heated cylinders of non-circular cross-section,  (the fluid flow being along the generators) . 


/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
